In [1]:
import scanpy as sc 
import pandas as pd

In [2]:
import csv
import gzip
import os
import scipy.io

In [3]:
outpath="/home/lugli/spuccio/Projects/SP039/FileH5AD_input/"

In [4]:
matrix_dir = "/home/lugli/spuccio/Projects/SP039/GSE173278_PMID_35303420/Raw/"
# read in MEX format matrix as table
mat = scipy.io.mmread(os.path.join(matrix_dir, "matrix.mtx.gz"))

In [5]:
features_path = os.path.join(matrix_dir, "features.tsv.gz")
feature_ids = [row[0] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

In [6]:
gene_names = [row[1] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]

In [7]:
# list of feature_types, e.g. 'Gene Expression'
feature_types = [row[1] for row in csv.reader(gzip.open(features_path, mode="rt"), delimiter="\t")]
barcodes_path = os.path.join(matrix_dir, "barcodes.tsv.gz")
barcodes = [row[0] for row in csv.reader(gzip.open(barcodes_path, mode="rt"), delimiter="\t")]

In [8]:
matrix = pd.DataFrame.sparse.from_spmatrix(mat)
matrix.columns = barcodes
matrix.insert(loc=0, column="feature_id", value=feature_ids)
matrix.insert(loc=0, column="gene", value=gene_names)
matrix.insert(loc=0, column="feature_type", value=feature_types)

In [9]:
matrix = matrix.drop_duplicates(['gene'])

In [10]:
matrix['GeneTempClean']= list(matrix['gene'].str.split(".",0, expand=True)[0])

In [11]:
matrix = matrix.set_index(['GeneTempClean'],drop=True)

In [12]:
matrix.head()

,feature_type,gene,feature_id,AAACCTGCACAACTGT-1,AAACCTGCACCAGCAC-1,AAACCTGGTCAGGACA-1,AAACGGGAGCTGCCCA-1,AAACGGGTCTGTGCAA-1,AAAGATGAGAGCTATA-1,AAAGATGCACATAACC-1,...,TTGGAACGTTGATTGC-99,TTGGCAAAGGGATCTG-99,TTGGCAACATATACGC-99,TTTACTGAGTATTGGA-99,TTTACTGGTCGCGTGT-99,TTTATGCAGGCACATG-99,TTTGCGCAGAAACCGC-99,TTTGCGCCACCACGTG-99,TTTGCGCGTACCGGCT-99,TTTGCGCGTTCCCTTG-99
GeneTempClean,,,,,,,,,,,,,,,,,,,,,
RP11-34P13,RP11-34P13.3,RP11-34P13.3,ENSG00000243485,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13,RP11-34P13.7,RP11-34P13.7,ENSG00000238009,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13,RP11-34P13.8,RP11-34P13.8,ENSG00000239945,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13,RP11-34P13.9,RP11-34P13.9,ENSG00000241599,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FO538757,FO538757.3,FO538757.3,ENSG00000279928,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
del matrix['feature_type']
del matrix['gene']
del matrix['feature_id']

In [14]:
adata = sc.AnnData(matrix.T)

/tmp/ipykernel_7209/3338772884.py:1: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(matrix.T)
/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [15]:
adata.var_names_make_unique()

In [16]:
meta = pd.read_csv("/home/lugli/spuccio/Projects/SP039/GSE173278_PMID_35303420/GSE173278_scRNAseq_all_cells_metadata.csv.gz")

In [17]:
meta.set_index(['cell_name'],drop=True,inplace=True)

In [18]:
adata.obs = meta

In [19]:
adata.obs['patient'].value_counts()

JK125    13306
JK142    12376
JK152    11947
JK156     9326
JK153     8987
JK124     8462
JK136     7362
JK163     5873
JK126     5496
JK134     3397
Name: patient, dtype: int64

In [20]:
adata.obs['source'].value_counts()

org     40726
tis     34024
cell    11782
Name: source, dtype: int64

In [21]:
adata = adata[adata.obs['source'].isin(['tis']),:]

In [22]:
adata = adata[adata.obs['source'].isin(['tis']),:]

In [23]:
JK124 = adata[adata.obs['patient'].isin(['JK124']),:]
JK125 = adata[adata.obs['patient'].isin(['JK125']),:]
JK126 = adata[adata.obs['patient'].isin(['JK126']),:]
JK134 = adata[adata.obs['patient'].isin(['JK134']),:]
JK136 = adata[adata.obs['patient'].isin(['JK136']),:]
JK142 = adata[adata.obs['patient'].isin(['JK142']),:]
JK152 = adata[adata.obs['patient'].isin(['JK152']),:]
JK153 = adata[adata.obs['patient'].isin(['JK153']),:]
JK156 = adata[adata.obs['patient'].isin(['JK156']),:]
JK163 = adata[adata.obs['patient'].isin(['JK163']),:]

In [24]:
JK124.obs = JK124.obs.reset_index(drop=True)
JK125.obs = JK125.obs.reset_index(drop=True)
JK126.obs = JK126.obs.reset_index(drop=True)
JK134.obs = JK134.obs.reset_index(drop=True)
JK136.obs = JK136.obs.reset_index(drop=True)
JK142.obs = JK142.obs.reset_index(drop=True)
JK152.obs = JK152.obs.reset_index(drop=True)
JK153.obs = JK153.obs.reset_index(drop=True)
JK156.obs = JK156.obs.reset_index(drop=True)
JK163.obs = JK163.obs.reset_index(drop=True)

In [25]:
JK124.obs.set_index("JK124-" + JK124.obs.index.astype(str),inplace=True)
JK125.obs.set_index("JK125-" + JK125.obs.index.astype(str),inplace=True)
JK126.obs.set_index("JK126-" + JK126.obs.index.astype(str),inplace=True)
JK134.obs.set_index("JK134-" + JK134.obs.index.astype(str),inplace=True)
JK136.obs.set_index("JK136-" + JK136.obs.index.astype(str),inplace=True)
JK142.obs.set_index("JK142-" + JK142.obs.index.astype(str),inplace=True)
JK152.obs.set_index("JK152-" + JK152.obs.index.astype(str),inplace=True)
JK153.obs.set_index("JK153-" + JK153.obs.index.astype(str),inplace=True)
JK156.obs.set_index("JK156-" + JK156.obs.index.astype(str),inplace=True)
JK163.obs.set_index("JK163-" + JK163.obs.index.astype(str),inplace=True)

In [26]:
del JK124.obs
del JK125.obs
del JK126.obs
del JK134.obs
del JK136.obs
del JK142.obs
del JK152.obs
del JK153.obs
del JK156.obs
del JK163.obs

In [27]:
JK124.obs['SampleID'] = "JK124"
JK124.obs['GEO_RNA'] = "GSM5264672"
JK124.obs['Cohort'] = "GSE173280"
JK124.obs['Type'] = "PrimaryTumor"  
JK124.obs['Subtype'] = "NotAvailable"
JK124.obs['Diagnosis'] = "Glioblastoma"
JK124.obs['Grade'] = "NotAvailable"
JK124.obs['Location'] = "Right_Temporal"
JK124.obs['IDH1_Status'] = "WT"
JK124.obs['EGFR_Status'] = "NotAvailable"
JK124.obs['CellCount'] = "6982"
JK124.obs['Sample'] = "Patient99"
JK124.obs['Sex'] = "M"
JK124.obs['Age'] = "65"
JK124.obs['Sorted'] = "CD45"

In [28]:
JK125.obs['SampleID'] = "JK125"
JK125.obs['GEO_RNA'] = "GSM5264680"
JK125.obs['Cohort'] = "GSE173280"
JK125.obs['Type'] = "PrimaryTumor"  
JK125.obs['Subtype'] = "NotAvailable"
JK125.obs['Diagnosis'] = "Glioblastoma"
JK125.obs['Grade'] = "NotAvailable"
JK125.obs['Location'] = "Right_Frontal"
JK125.obs['IDH1_Status'] = "WT"
JK125.obs['EGFR_Status'] = "NotAvailable"
JK125.obs['CellCount'] = "3169"
JK125.obs['Sample'] = "Patient100"
JK125.obs['Sex'] = "F"
JK125.obs['Age'] = "84"
JK125.obs['Sorted'] = "CD45"

In [29]:
JK126.obs['SampleID'] = "JK126"
JK126.obs['GEO_RNA'] = "GSM5264680"
JK126.obs['Cohort'] = "GSE173280"
JK126.obs['Type'] = "PrimaryTumor"  
JK126.obs['Subtype'] = "NotAvailable"
JK126.obs['Diagnosis'] = "Glioblastoma"
JK126.obs['Grade'] = "NotAvailable"
JK126.obs['Location'] = "Left_Frontal"
JK126.obs['IDH1_Status'] = "WT"
JK126.obs['EGFR_Status'] = "NotAvailable"
JK126.obs['CellCount'] = "2837"
JK126.obs['Sample'] = "Patient101"
JK126.obs['Sex'] = "M"
JK126.obs['Age'] = "58"
JK126.obs['Sorted'] = "CD45"

In [30]:
JK134.obs['SampleID'] = "JK134"
JK134.obs['GEO_RNA'] = "GSM5264680"
JK134.obs['Cohort'] = "GSE173280"
JK134.obs['Type'] = "PrimaryTumor"  
JK134.obs['Subtype'] = "NotAvailable"
JK134.obs['Diagnosis'] = "Glioblastoma"
JK134.obs['Grade'] = "NotAvailable"
JK134.obs['Location'] = "Right_Frontal"
JK134.obs['IDH1_Status'] = "WT"
JK134.obs['EGFR_Status'] = "NotAvailable"
JK134.obs['CellCount'] = "2346"
JK134.obs['Sample'] = "Patient102"
JK134.obs['Sex'] = "F"
JK134.obs['Age'] = "58"
JK134.obs['Sorted'] = "CD45"

In [31]:
JK136.obs['SampleID'] = "JK136"
JK136.obs['GEO_RNA'] = "GSM5264680"
JK136.obs['Cohort'] = "GSE173280"
JK136.obs['Type'] = "PrimaryTumor"  
JK136.obs['Subtype'] = "NotAvailable"
JK136.obs['Diagnosis'] = "Glioblastoma"
JK136.obs['Grade'] = "NotAvailable"
JK136.obs['Location'] = "Left_Temporal"
JK136.obs['IDH1_Status'] = "WT"
JK136.obs['EGFR_Status'] = "NotAvailable"
JK136.obs['CellCount'] = "4372"
JK136.obs['Sample'] = "Patient103"
JK136.obs['Sex'] = "F"
JK136.obs['Age'] = "57"
JK136.obs['Sorted'] = "CD45"

In [32]:
JK142.obs['SampleID'] = "JK142"
JK142.obs['GEO_RNA'] = "GSM5264680"
JK142.obs['Cohort'] = "GSE173280"
JK142.obs['Type'] = "PrimaryTumor"  
JK142.obs['Subtype'] = "NotAvailable"
JK142.obs['Diagnosis'] = "Glioblastoma"
JK142.obs['Grade'] = "NotAvailable"
JK142.obs['Location'] = "Right_Temporal"
JK142.obs['IDH1_Status'] = "WT"
JK142.obs['EGFR_Status'] = "NotAvailable"
JK142.obs['CellCount'] = "4553"
JK142.obs['Sample'] = "Patient104"
JK142.obs['Sex'] = "F"
JK142.obs['Age'] = "66"
JK142.obs['Sorted'] = "CD45"

In [33]:
JK152.obs['SampleID'] = "JK152"
JK152.obs['GEO_RNA'] = "GSM5264680"
JK152.obs['Cohort'] = "GSE173280"
JK152.obs['Type'] = "PrimaryTumor"  
JK152.obs['Subtype'] = "NotAvailable"
JK152.obs['Diagnosis'] = "Glioblastoma"
JK152.obs['Grade'] = "NotAvailable"
JK152.obs['Location'] = "Right_Parietal"
JK152.obs['IDH1_Status'] = "WT"
JK152.obs['EGFR_Status'] = "NotAvailable"
JK152.obs['CellCount'] = "2625"
JK152.obs['Sample'] = "Patient105"
JK152.obs['Sex'] = "M"
JK152.obs['Age'] = "46"
JK152.obs['Sorted'] = "CD45"

In [34]:
JK153.obs['SampleID'] = "JK153"
JK153.obs['GEO_RNA'] = "GSM5264680"
JK153.obs['Cohort'] = "GSE173280"
JK153.obs['Type'] = "PrimaryTumor"  
JK153.obs['Subtype'] = "NotAvailable"
JK153.obs['Diagnosis'] = "Glioblastoma"
JK153.obs['Grade'] = "NotAvailable"
JK153.obs['Location'] = "Left_Parietal"
JK153.obs['IDH1_Status'] = "WT"
JK153.obs['EGFR_Status'] = "NotAvailable"
JK153.obs['CellCount'] = "3277"
JK153.obs['Sample'] = "Patient106"
JK153.obs['Sex'] = "F"
JK153.obs['Age'] = "66"
JK153.obs['Sorted'] = "CD45"

In [35]:
JK156.obs['SampleID'] = "JK156"
JK156.obs['GEO_RNA'] = "GSM5264680"
JK156.obs['Cohort'] = "GSE173280"
JK156.obs['Type'] = "PrimaryTumor"  
JK156.obs['Subtype'] = "NotAvailable"
JK156.obs['Diagnosis'] = "Glioblastoma"
JK156.obs['Grade'] = "NotAvailable"
JK156.obs['Location'] = "Left_Temporal"
JK156.obs['IDH1_Status'] = "WT"
JK156.obs['EGFR_Status'] = "NotAvailable"
JK156.obs['CellCount'] = "1548"
JK156.obs['Sample'] = "Patient107"
JK156.obs['Sex'] = "M"
JK156.obs['Age'] = "67"
JK156.obs['Sorted'] = "CD45"

In [36]:
JK163.obs['SampleID'] = "JK163"
JK163.obs['GEO_RNA'] = "GSM5264680"
JK163.obs['Cohort'] = "GSE173280"
JK163.obs['Type'] = "PrimaryTumor"  
JK163.obs['Subtype'] = "NotAvailable"
JK163.obs['Diagnosis'] = "Glioblastoma"
JK163.obs['Grade'] = "NotAvailable"
JK163.obs['Location'] = "Right_Frontal"
JK163.obs['IDH1_Status'] = "WT"
JK163.obs['EGFR_Status'] = "NotAvailable"
JK163.obs['CellCount'] = "2315"
JK163.obs['Sample'] = "Patient108"
JK163.obs['Sex'] = "M"
JK163.obs['Age'] = "64"
JK163.obs['Sorted'] = "CD45"

In [37]:
JK124.write(outpath+"JK124.h5ad")
JK125.write(outpath+"JK125.h5ad")
JK126.write(outpath+"JK126.h5ad")
JK134.write(outpath+"JK134.h5ad")
JK136.write(outpath+"JK136.h5ad")
JK142.write(outpath+"JK142.h5ad")
JK152.write(outpath+"JK152.h5ad")
JK153.write(outpath+"JK153.h5ad")
JK156.write(outpath+"JK156.h5ad")
JK163.write(outpath+"JK163.h5ad")